In [ ]:
!pip install -U finance-datareader
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')

In [ ]:
import sqlite3
conn = sqlite3.connect('./db.stock')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS article (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, time TEXT, title TEXT, press TEXT , stock TEXT, posi_nega TEXT)")

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# https://jsikim1.tistory.com/143
from datetime import datetime , timedelta
from dateutil.relativedelta import relativedelta 

now = datetime.now()
gap = now - relativedelta(years=1)
# gap = now - timedelta(days=1)

now = str(now)[0:10]
gap = str(gap)[0:10]

dt_index = pd.date_range(start=gap, end=now)
dt_list = dt_index.strftime("%Y%m%d").tolist()

for j in dt_list:

  date_cnt_uri = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3=402&date='+j+'&page=100'
  date_cnt_target = date_cnt_uri
  date_cnt_req  = requests.get(date_cnt_target)
  date_cnt_soup = BeautifulSoup(date_cnt_req.content,'html.parser')
  date_cnt_page = int(date_cnt_soup.select('td.on > a ')[0].get_text())
  
  uri = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3=402&date='+j+'&page='

  for page in range(1,date_cnt_page+1):    

    target = uri+str(page)
    req  = requests.get(target)
    soup = BeautifulSoup(req.content,'html.parser')
    datas = soup.select('#contentarea_left > ul.realtimeNewsList')

    for content in datas:
      
      titles = content.select(' li > dl > dd.articleSubject')
      article_date =  content.select('li > dl > dd.articleSummary > span.wdate ')
      article_press =  content.select('li > dl > dd.articleSummary > span.press ') 

      article_sum = list()

      for i in range(0,len(titles)-1):
        article_data = list()
        
        data_date = article_date[i].get_text(" ",strip=True)[0:10]
        data_time = article_date[i].get_text(" ",strip=True)[11:17]
        data_press = article_press[i].get_text(" ",strip=True)
        data_title = titles[i].get_text(" ",strip=True)

        c.execute("INSERT INTO article( date , time , press , title, stock , posi_nega ) VALUES(?,?,?,?,?,?)",( data_date,data_time,data_press,data_title,'stock','posi_nega'))
        conn.commit()

c.close()

In [ ]:
conn = sqlite3.connect('/content/db.stock')

c_title = conn.cursor()
c_id = conn.cursor()
c_update = conn.cursor()

c_title.execute("SELECT title FROM article ")
c_id.execute("SELECT * FROM article ")

sql_title = c_title.fetchall()
sql_id = c_id.fetchall()

title_list = [list[0] for list in sql_title ]
id_list = [list[0] for list in sql_id ]

df_krx_list = df_krx['Name'].tolist()

for k in range(0,len(title_list)):
  for l in range(0,int(len(title_list[k].split()))):
    keyword = title_list[k].split()[l]
    if keyword in df_krx_list:
      sql_update = 'update article set stock = "' + keyword +'" where id = ' + str(sql_id[k][0])
      c_update.execute(sql_update)
      conn.commit()

In [ ]:
conn = sqlite3.connect('/content/db.stock')
c_select = conn.cursor()
final = c_select.execute("SELECT * FROM article where stock != 'stock' order by date desc , time desc")
df = pd.DataFrame(final)
df.columns= ['id', 'date' , 'time' , 'title', 'press' ,'stock' , 'posi_nega' ]
df

,id,date,time,title,press,stock,posi_nega
0,27590,2021-08-12,11:38,크래프톤 충격에 갈 곳 잃은 게임주···엔씨소프트 목표주가 뚝,아시아경제,크래프톤,posi_nega
1,27605,2021-08-12,10:20,[특징주]메모리 반도체 우려에 원익IPS 등 장비주도 약세,파이낸셜뉴스,원익IPS,posi_nega
2,27607,2021-08-12,10:08,"베스파 신작 '타임디펜더스', 24일 일본 정식 출시 확정",파이낸셜뉴스,베스파,posi_nega
3,27608,2021-08-12,09:54,"위지윅스튜디오 자회사 위즈온센, 中 ATW와 파트너십 계약",파이낸셜뉴스,위지윅스튜디오,posi_nega
4,27614,2021-08-12,09:28,"[특징주] MSCI 편입 소식에 SKITE, 에코프로비엠 급등",머니투데이,에코프로비엠,posi_nega
...,...,...,...,...,...,...,...
2993,62,2020-08-12,10:02,"한국증권금융 꿈나눔재단, 어려움 겪는 청소년 대상 장학금 지원",매일경제,대상,posi_nega
2994,66,2020-08-12,09:40,"셀트리온 ""국내 업체와 함께 개발한 코로나19 진단키트 美시장 출시""",파이낸셜뉴스,셀트리온,posi_nega
2995,68,2020-08-12,09:34,"[특징주]SK디스커버리, SK바이오사이언스 백신 생산 시설 증설 소식에 강세",파이낸셜뉴스,SK바이오사이언스,posi_nega
2996,86,2020-08-12,08:22,삼성바이오로직스 新공장 기대↑…목표가 '100만원'-KTB,파이낸셜뉴스,삼성바이오로직스,posi_nega
